# Simulation II: Collision Detection with HPP-FCL

![](./lots_of_objs.png)

# Part I - A (very) simple kinematic simulator with HPP-FCL
The goal of this part is to show how the narrow phase collision detection work in HPP-FCL and ultimately build a very simple kinematic simulator. 
To do so, we will create a bunch of objects with HPP-FCL, have their collisions checked and update their velocities based on the normal of the collision (if any).

We will see in part II of this notebook how to interface with pinocchio.
Finally, we will see in `Simulation III` how to properly add physically-accurate contact resolutions.

In this tutorial, we will mainly need HPP-FCL.
We will also use Pinocchio's `SE3` module to update rotations of moving shapes.
Finally, we will also be using meshcat for visualiztion.

In [ ]:
import magic_donotload
import hppfcl
import pinocchio as pin
import numpy as np
import meshcat
from typing import List

## 1.1 - Create shapes with HPP-FCL and visualize them

HPP-FCL supports lots of primitives shapes such as boxes, spheres, capsules, ellipsoids, cones, cylinders as well as convex meshes (clouds of points and triangles which define a convex closed surface).
HPP-FCL also supports non-convex meshes (any cloud of points), height-fields, octrees etc.

In HPP-FCL, we make the distinction between `CollisionGeometry` and `CollisionObject`.
A `CollisionObject` is a `CollisionGeometry` + a 3D placement (an element of `SE3`).

In [ ]:
# A capsule is a CollisionGeometry
capsule_geometry = hppfcl.Capsule(0.1, 0.2)
M = pin.SE3.Random()
# We create a CollisionObject out of a geometry and a placement
capsule = hppfcl.CollisionObject(capsule_geometry, M)

print(capsule.getNodeType())
print("Rotation matrix: \n", capsule.getRotation())
print("Translation vector: \n", capsule.getTranslation())

Let's now visualize that.

To keep things simple, we provide some utilities to visualize our collision objects.
We create a very simple object `Scene` which will hold some basic information about our world:

In [ ]:
# This code is only for illustration, `Scene` will be replaced by `AgimusScene`
class Scene:
    collision_objects: List[hppfcl.CollisionObject]
    viewer: meshcat.Visualizer
    mc_shapes: List[meshcat.geometry.Geometry]

    def __init__(self):
        pass

    def render_scene(self):
        pass

    def clear_scene(self):
        pass

Let's import an already filled-out `AgimusScene`, which defines some useful rendering functions, and visualize our shapes:

In [ ]:
from utils_render import AgimusScene, load_convex 

# This will be our scene
scene = AgimusScene()

capsule_geom = hppfcl.Capsule(0.1, 0.2)
M = pin.SE3.Random()
color = np.random.rand(3)
scene.register_object(capsule_geom, M, color)

convex_geom = load_convex("./assets/mesh.stl")
M = pin.SE3.Random()
color = np.random.rand(3)
scene.register_object(convex_geom, M, color)

# Initialize scene renderer
scene.init_renderer()

# Render scene
scene.render_scene()


In [ ]:
# Visualize the meshcat viewer in the notebook
scene.viewer.jupyter_cell()

## 1.2 - Narrow phase collision check

In a real-life scenario, we always start by running a broad phase collision check to see if the bounding boxes of ours shapes intersect. 
If so, we run a narrow phase collision check on the shapes.

For the sake of this practical session, we will do the opposite and look at the narrow phase first.

To run a narrow phase collision check between two objects, we first need to create a `CollisionRequest` and a `CollisionResult`.
These two classes hold the parameters and the results of our collision check.

In [ ]:
colreq = hppfcl.CollisionRequest()
colres = hppfcl.CollisionResult()

Let's run the collision check:

In [ ]:
is_collision = hppfcl.collide(scene.collision_objects[0],
                              scene.collision_objects[1],
                              colreq, colres)
print("Is collision?: ", is_collision)
hasattr(scene.viewer, 'jupyter_cell') and scene.viewer.jupyter_cell()

Since we have selected random 3D placements for our shapes, it's likely the result of the previous cell was that the shapes are not in collision.

However, it's possible to define a `security_margin` in our `CollisionRequest`.
If it has a positive value, a collision will be registered as long as the distance between the shapes is less than the security margin:

In [ ]:
# Let's clear our previous result
colres.clear()
# Let's set the security margin in our request to a high number
colreq.security_margin = 100

# Let's run the collision detection again
is_collision = hppfcl.collide(scene.collision_objects[0],
                              scene.collision_objects[1],
                              colreq, colres)
print("Is collision?: ", is_collision)

We can view the results and observe the separation vector between our shapes:

In [ ]:
scene.visualize_separation_vector(colres)
scene.viewer.jupyter_cell()

As we see, although our shapes visually don't touch, , the narrow phase now considers the shapes to be in contact. 
Again, this is due to the high `security_margin` we imposed in our collision request.

You can access the penetration depth, normal, and contact points as so:

In [ ]:
contact: hppfcl.Contact = colres.getContact(0)
normal = contact.normal
cp1 = contact.getNearestPoint1()
cp2 = contact.getNearestPoint2()
v = cp2 - cp1
depth = contact.penetration_depth

print(f"normal: {normal}")
print(f"cp1: {cp1}")
print(f"cp2: {cp2}")
print(f"separaction vector: {v}")
print(f"the normal is the normalized separation vector\n    v/||v||: {v / np.linalg.norm(v)},\n    normal: {normal}")
print(f"penetration depth: {depth}")

Note: the penetration depth is the "true" distance between the collision objects (without taking into consideration the security margin).
For objects to be considered in considered in contact by the narrow phase, we therefore have to have:
 `contact.penetration_depth - colreq.security_margin <= 0.`

Exercice: make distant objects touch.

Now that we have access to the separation vector `v = cp1 - cp2`, we can translate `scene.collision_objects[1]` along the separation vector until the two objects collide with each other.

In [ ]:
# %load -r 6-18 aws_collision_sols.py
def make_two_objects_touch(shape1: hppfcl.CollisionObject, shape2: hppfcl.CollisionObject):
    # TODO: re-write this function in order to translate shape2 along the separation vector
    # so that shape1 and shape2 touch
    colreq = hppfcl.CollisionRequest()
    colres = hppfcl.CollisionResult()
    hppfcl.collide(shape1, shape2, colreq, colres)

    # Note: we can get and set the translation of a shape as so:
    shape2.getTranslation()
    shape2.setTranslation(np.random.rand(3))

In [ ]:
make_two_objects_touch(scene.collision_objects[0], scene.collision_objects[1])
scene.render_scene()
scene.viewer.jupyter_cell()

## 1.3 - Building a (very) simple kinematic simulator

Let's create more complex scene than what we had so far:

In [ ]:
from utils_render import create_complex_scene

# The scene is made of a box (6 walls) with a bunch of objects inside
shapes, transforms, scene = create_complex_scene()
n_walls = 6

# Initialize scene renderer
scene.init_renderer()

# render the scene
scene.render_scene()

num_collision_objects = len(scene.collision_objects)
print("Number of collision objects", num_collision_objects)
num_possible_collision_pairs = (int)(len(scene.collision_objects)*(len(scene.collision_objects) - 1)/2)
print("Number of possible collision pairs: ", num_possible_collision_pairs)

scene.viewer.jupyter_cell()

Exercice: let's try to build a very simple simulation: objects only have a velocity (no acceleration or inertia); have their position updated according to their velocity.
It's up to you to decide how the contacts are resolved once collisions are found.
You can be as creative as you want about it!

We'll see in Simulation III how to make things physically accurate.

In [ ]:
# Starting velocities of each shape
velocities = []
for i in range(num_collision_objects - n_walls):
    v = np.random.rand(6) * 0.25
    velocities.append(v)
# We don't want the walls to move
for i in range(n_walls):
    velocities.append(np.zeros(6))
 
# We can use this handy function to reset the placemements of our simulated objects to their initial placements
def reset_objects_placements(scene, transforms):
    for s in range(len(scene.collision_objects)):
        scene.collision_objects[s].setTransform(transforms[s])
reset_objects_placements(scene, transforms)


In [ ]:
scene.viewer.jupyter_cell()

Note: I suggest you run the two cells below with `CTRL+Enter` to not jump to the cell below it. By doing that you can launch the code in the simulator cell and see the results in the cell above.

- You can use the first cell below to reset the simulated objects to their initial placements.
- You can use the second cell below to run the simulation.

In [ ]:
reset_objects_placements(scene, transforms)
scene.render_scene()

In [ ]:
# %load -r 34-84 aws_collision.py

# Simulation loop
# You can increase the time horizon of the simulation
T = 100
dt = 0.1 # timestep

# Collision request and result needed for the narrow phase
colreq = hppfcl.CollisionRequest()
colres = hppfcl.CollisionResult()
for t in range(T):
    # Render the current scene
    scene.render_scene()

    # Loop through all collision pairs
    for i in range(0, num_collision_objects-1):
        for j in range(i+1, num_collision_objects):
            # If both object are walls, we don't really care about checking their collision
            if i < num_collision_objects - n_walls or j < num_collision_objects - n_walls:
                colres.clear()
                is_colliding = hppfcl.collide(scene.collision_objects[i], scene.collision_objects[j], colreq, colres)
                # -----------------------------------------------------
                # TODO: fill what is inside this "if" statement to correct
                # the velocities of the two objects found # to be in collision
                # -----------------------------------------------------
                if (is_colliding):
                    pass

    # Update the placements of the shapes based on their velocities
    for i in range(num_collision_objects - n_walls):
        M = scene.collision_objects[i].getTransform()
        # I will be using the first 3 elements of velocities[i] to apply a linear velocity to M
        # in the world frame.
        # And I will be using the last 3 elements of velocities[i] to apply an angular velocity to M
        # in its local frame.
        v1 = np.zeros(6)
        v2 = np.zeros(6)
        v1[:3] = velocities[i][:3]
        v2[3:] = velocities[i][3:]
        M = pin.exp6(v1*dt) * M * pin.exp6(v2*dt)
        scene.collision_objects[i].setTransform(M)


# Part II - Pinocchio + HPP-FCL

HPP-FCL is integrated in Pinocchio which uses it under the hood for broad and narrow phase collision detection.
What's nice is that all the quantities that we have seen so far (geometric shapes, collision requests/results, normals, contact points...) are also exposed by pinocchio.

In this section we load a panda robot and make it play a very simple collision avoidance task.

In [ ]:
from aws_collision_sols import create_panda
model, geom_model, visual_model = create_panda()

The `geom_model` of the robot is the model used for collision detection.
By default pinocchio creates collision pairs for every pair of links `(i, j)` where `j!=i` and `j!=i+1` (a link cannot collide with itself and with its direct neighbor).

It's too easy to find a collision-free configuration for the robot itself so let's a scene with a bunch of objects around the robot:

In [ ]:
NUM_GEOM_PANDA = len(geom_model.geometryObjects)
def add_balls_to_geom_model(geom_model, joint, n_ball, a, M, visual=False):
    ball_shape = hppfcl.Sphere(a/2)
    geom_ball = pin.GeometryObject( "ball_" + str(n_ball), joint, joint, M, ball_shape)
    geom_ball.meshColor = np.array([1.0, 0.2, 0.2, 1.0])
    ball_id = geom_model.addGeometryObject(geom_ball)
    if not visual:
        for id in range(NUM_GEOM_PANDA - 1):
            col_pair = pin.CollisionPair(id, ball_id)
            geom_model.addCollisionPair(col_pair)
    return geom_model

xs = np.arange(-5, 7, 2)/6.
ys = np.arange(-5, 7, 2)/6.
zs = np.arange(-5, 7, 2)/6.
i = 0
for x in xs:
    for y in ys:
        for z in zs:
            M = pin.SE3.Random()
            M.translation = np.array([x, y, z])
            add_balls_to_geom_model(geom_model, 0, i, 0.1, M)
            add_balls_to_geom_model(visual_model, 0, i, 0.1, M, True)
            i += 1

data = model.createData()
geom_data = geom_model.createData()

In [ ]:
Viewer = pin.visualize.MeshcatVisualizer
viz = Viewer(model, geom_model, visual_model)
viz.initViewer(loadModel=True)
viz.display(pin.neutral(model))
viz.viewer.jupyter_cell()

Exercice: find a configuration of panda that is collision free.

In [ ]:
# %load -r 20-30 aws_collision_sols.py
def find_free_collision_configuration(model, data, geom_model, geom_data):
    # TODO
    return q

In [ ]:
q = find_free_collision_configuration(model, data, geom_model, geom_data)
viz.display(q)

Question: given a start configuration `qstart` and an end configuration `qend`, how generate a collision-free path between these two configurations?